In [2]:
import plotly.graph_objects as go
from dash import Input, Output, dcc, html
import pandas as pd

#from app.config import regions_path
#from app.app_factory import make_main_app, make_app_region
#from app.services.data_loader import create_gdf

# Загрузка CSV файла с аналитическими данными
csv_data = pd.read_csv("app/data/analytic/data.csv")

In [3]:
csv_data

,region_name,region,value,percent_change,budget_millions,population_change,details
0,Жуковский,Московская область,4.37,-19,1250.5,-2.3,"Детали для Жуковский: изменение на -19%, приор..."
1,Сочи,Краснодарский край,9.56,9,2850.8,5.7,"Детали для Сочи: изменение на 9%, приоритет вы..."
2,Невинномысск,Ставропольский край,7.59,17,890.2,3.1,"Детали для Невинномысск: изменение на 17%, при..."
3,Кисловодск,Ставропольский край,6.39,-19,650.4,-1.8,"Детали для Кисловодск: изменение на -19%, прио..."
4,Гатчина,Ленинградская область,2.40,43,420.7,8.2,"Детали для Гатчина: изменение на 43%, приорите..."
5,Рыбинск,Ярославская область,2.40,39,380.9,4.5,"Детали для Рыбинск: изменение на 39%, приорите..."
6,Барнаул,Алтайский край,1.52,0,950.3,1.2,"Детали для Барнаул: изменение на 0%, приоритет..."
7,Чита,Забайкальский край,8.80,12,1450.6,2.8,"Детали для Чита: изменение на 12%, приоритет в..."
8,Улан-Удэ,Республика Бурятия,6.41,37,780.1,6.9,"Детали для Улан-Удэ: изменение на 37%, приорит..."
9,Уссурийск,Приморский край,7.37,1,1100.4,0.5,"Детали для Уссурийск: изменение на 1%, приорит..."


In [2]:
gdf = create_gdf(regions_path)

app_main = make_main_app(gdf)
app_region = make_app_region(app_main)

In [3]:
@app_main.callback(
    [Output("choropleth", "figure"), Output("region-link", "href")],
    [Input("year-slider", "value"), Input("choropleth", "clickData")],
)
def update_main_map(year, clickData):
    fig = go.Figure(
        go.Choroplethmapbox(
            geojson=gdf.geometry.__geo_interface__,
            locations=gdf.index,
            z=gdf["value"],
            colorscale="Viridis",
            hovertemplate="<b>%{text}</b><br>Значение: %{z:.0f}<extra></extra>",
            text=gdf["name"],
            ids=gdf.index,  # Для клика
        )
    )

    region_url = "/region-dashboard"
    if clickData:
        clicked_idx = clickData["points"][0]["pointIndex"]
        region_name = gdf.iloc[clicked_idx]["name"]
        # ✅ ССЫЛКА НА ДАШБОРД РЕГИОНА С ПАРАМЕТРОМ
        region_url = f"/region-dashboard?region={region_name}"

        # Зум на регион
        region_geom = gdf.iloc[clicked_idx].geometry
        centroid = region_geom.centroid
        fig.update_layout(
            mapbox_zoom=5, mapbox_center={"lat": centroid.y, "lon": centroid.x}
        )

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=3 if not clickData else 5,
        mapbox_center={"lat": 60, "lon": 90},
        height=700,
    )
    return fig, region_url


@app_region.callback(
    Output("region-dashboard-content", "children"), Input("url", "search")
)
def display_region_dashboard(url_search):
    # Извлекаем название региона из URL
    if url_search and "region=" in url_search:
        region_name = url_search.split("region=")[1].split("&")[0].replace("+", " ")
        region_data = gdf[gdf["name"] == region_name]

        if len(region_data) > 0:
            row = region_data.iloc[0]
            return html.Div(
                [
                    html.H1(f"Дашборд: {region_name}", style={"color": "#007bff"}),
                    html.Div(
                        [
                            html.Div(
                                [
                                    html.H3("Основные показатели"),
                                    html.P(f"Значение: {row['value']:.0f}"),
                                    html.P(f"Население: {row['population']:.0f}"),
                                    html.P(f"Площадь: {row['area']:.0f} км²"),
                                    html.P(f"ВВП: {row['gdp']:.0f} млрд руб."),
                                ],
                                className="col-md-6",
                            ),
                            html.Div(
                                [
                                    html.H3("Карта региона"),
                                    dcc.Graph(
                                        figure=go.Figure(
                                            go.Choroplethmapbox(
                                                geojson=region_data.geometry.__geo_interface__,
                                                locations=region_data.index,
                                                z=[row["value"]],
                                                colorscale="Viridis",
                                                showscale=False,
                                            )
                                        ).update_layout(
                                            mapbox_zoom=7,
                                            mapbox_center={
                                                "lat": row.geometry.centroid.y,
                                                "lon": row.geometry.centroid.x,
                                            },
                                        )
                                    ),
                                ],
                                className="col-md-6",
                            ),
                        ],
                        className="row",
                    ),
                    html.A(
                        "← Назад к карте России",
                        href="/",
                        target="_blank",
                        style={
                            "margin": "20px",
                            "padding": "10px",
                            "background": "#6c757d",
                            "color": "white",
                            "textDecoration": "none",
                            "borderRadius": "5px",
                        },
                    ),
                ],
                style={"padding": "40px", "maxWidth": "1200px", "margin": "auto"},
            )

    return html.Div(
        "Выберите регион на главной карте",
        style={"textAlign": "center", "padding": "100px"},
    )

In [5]:
app_main.run(debug=True, port=8080, host="0.0.0.0")

/tmp/ipykernel_186186/2281234708.py:7: DeprecationWarning:

*choroplethmapbox* is deprecated! Use *choroplethmap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

